In [1]:
# set conda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# install psi4
!conda install -y psi4 python=3.7 -c psi4

# set path
import sys
sys.path.append("/usr/local/lib/python3.7/site-packages/")

--2022-04-18 09:57:21--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2022-04-18 09:57:22--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75660608 (72M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  72.16M   153MB/s    in 0.5s    

2022-04-18 09:57:22 (153 MB/s) - ‘Miniconda3-latest-Linux

In [2]:
# import Psi4
import psi4
# check Psi4 version
print(psi4.__version__)

1.5


In [3]:
psi4.core.set_output_file("test.out")

mol = psi4.geometry("""
0 1
 C   -1.073986885437   -1.170578359942    0.000002755738
 C    0.953021017198    0.140745143377   -0.000003061297
 C    0.196029366481    1.360797354321   -0.000001202096
 C   -1.155622586106    1.236381947214    0.000002880952
 C   -1.818434996700   -0.031125507024    0.000005467599
 H   -1.510894241728   -2.166796480067    0.000004269828
 H    0.691726656323    2.319296967258   -0.000002742435
 H   -1.770734621411    2.131633416218    0.000004678262
 H   -2.895472126141   -0.076640107909    0.000009377672
 O    2.177846935050   -0.264655641303   -0.000003962413
 H    1.432324292904   -1.412270186206   -0.000002894660
 N    0.288380859812   -1.071403950594   -0.000002249524            
""")

psi4.set_output_file('Sty_Wiberg.txt')
scf_e, wfn = psi4.energy("B3LYP/6-31G(d)", return_wfn = True)
print('energy at the {} level of theory:\t{:.4f}'.format("B3LYP/6-31G(d)", scf_e))

energy at the B3LYP/6-31G(d) level of theory:	-323.4536


In [4]:
psi4.oeprop(wfn, 'WIBERG_LOWDIN_INDICES')
!cat Sty_Wiberg.txt


Scratch directory: /tmp/

Scratch directory: /tmp/

*** tstart() called on 41d1abdb89a6
*** at Mon Apr 18 09:59:49 2022

   => Loading Basis Set <=

    Name: 6-31G(D)
    Role: ORBITAL
    Keyword: BASIS
    atoms 1-5     entry C          line   111 file /usr/local/share/psi4/basis/6-31g_d_.gbs 
    atoms 6-9, 11 entry H          line    44 file /usr/local/share/psi4/basis/6-31g_d_.gbs 
    atoms 10      entry O          line   145 file /usr/local/share/psi4/basis/6-31g_d_.gbs 
    atoms 12      entry N          line   128 file /usr/local/share/psi4/basis/6-31g_d_.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RKS Reference
                        1 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point g

In [5]:
import numpy as np
import pandas as pd
df = pd.DataFrame()

psi4.oeprop(wfn, 'MULLIKEN_CHARGES')

df['B3LYP/6-31G(d)'] = np.array(wfn.atomic_point_charges())
print (df)

    B3LYP/6-31G(d)
0         0.092309
1         0.593658
2        -0.177855
3        -0.097656
4        -0.183515
5         0.167391
6         0.157628
7         0.152453
8         0.142570
9        -0.637658
10        0.401752
11       -0.611076


In [6]:
#psi4.oeprop(wfn,'DIPOLE')
dipole_vec = psi4.variable('scf dipole')
dipole = np.sqrt(np.sum(dipole_vec ** 2))
print(f'{dipole:.3f} [D]')

1.403 [D]
